In [1]:
import pandas as pd

import requests 

import os

import json

import googleapiclient.discovery
import googleapiclient.errors

import numpy as np

In [2]:
chiave_yt = json.load(open('key.json', 'r'))['youtube']['key'][2]
youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=chiave_yt)

In [3]:
#insert initial quota value
quota = 0

In [4]:
df_videos = pd.read_csv('data/get_youtuber_videos_output/youtuber_videos.csv')

In [5]:
def get_infos(response):
    info_comment_list=[]
    video_id=next_page_token = thread_comment_id = top_level_comment_id = author_display_name = author_channel_id = author_channel_id = text_original=None
    try:
        next_page_token = response['nextPageToken']
    except KeyError as e:
        #print(response)
        next_page_token = None
        print('Page token non esistente')
    items = response['items']
    for item in items:
        thread_comment_id = item['id']
        try:
            video_id = item['snippet']['videoId']
            top_level_comment = item['snippet']['topLevelComment']
            top_level_comment_id = top_level_comment['id']
            top_level_comment_snippet = top_level_comment['snippet']
            author_display_name = top_level_comment_snippet['authorDisplayName']
            text_original = top_level_comment_snippet['textOriginal']
            author_channel_id = top_level_comment_snippet['authorChannelId']['value']
        except KeyError as e:
            print(e.args[0])
            print('Problem with commentThread {}'.format(thread_comment_id))
        info_comment_list.append([video_id,thread_comment_id,top_level_comment_id,author_display_name,author_channel_id,text_original])
    return next_page_token, info_comment_list

In [6]:
#############################################################################

In [7]:
#round 1
#la prossima volta togli e decommenta quello sotto

In [8]:
last_csv = int(np.load('data/get_comment_threads_output/last_csv.npy'))
next_page_token = str(np.load('data/get_comment_threads_output/next_page_token.npy'))
last_video_idx = int(np.load('data/get_comment_threads_output/video_to_start.npy'))
videos_not_finished = (np.load('data/get_comment_threads_output/videos_not_finished.npy')).tolist()
videos_disabled_comments = (np.load('data/get_comment_threads_output/videos_disabled_comments.npy')).tolist()
print("Starting with video #{}, next_page_token: {}".format(last_video_idx,next_page_token))
print("Writing on {} csv".format(last_csv+1))

Starting with video #999, next_page_token: QURTSl9pM211d1UzelB0bXhkZnY2Wm9DdXo2LU9jZE56QmRKN3Fxa1V2R2VzWWpnYlVlaEVyNzFWa3RwWGg0aFBseUlCQXR4aEozZmVDcldpc1MwUkIydzlQNjZZaDludno2emU5QTJGeXRlVlpYV21FWkZwdDBtLUlKUjMxV3pwQVVEakJlaDlRZXBnbDQ=
Writing on 3 csv


In [9]:
df_videos_0 = df_videos[last_video_idx:]

In [10]:
#per i video con più di 10000 commenti, salvo l'indice del video con il next page token

In [11]:
comments = []
comments_count = 0

In [12]:
df_videos_0

,channel_id,video_id
999,UCmOKlMHHTV4DX3t21j3itPw,puxXSTyYh7s
1000,UCmOKlMHHTV4DX3t21j3itPw,GzpJ8RYlrM4
1001,UCmOKlMHHTV4DX3t21j3itPw,5mT3SYWwK-A
1002,UCmOKlMHHTV4DX3t21j3itPw,q7lRWRXBUds
1003,UCmOKlMHHTV4DX3t21j3itPw,j7VgLCPLonY
...,...,...
2588,UCq7EY7H2XF6TV7Z5mLv-aNg,bm-3Qrr7EUA
2589,UCq7EY7H2XF6TV7Z5mLv-aNg,hkxRjDS_Cjw
2590,UCq7EY7H2XF6TV7Z5mLv-aNg,8kElrRAa1ws
2591,UCq7EY7H2XF6TV7Z5mLv-aNg,7x6tX5T7LOs


In [13]:
for video in df_videos_0['video_id'].values:
    print('________________________________________________________')
    print('{} video: {}'.format(last_video_idx, video))
    video_comments_count = 0
    while next_page_token != None and video_comments_count<10000:
        print('Reached quotas: {}'.format(quota))
        if quota > 9900:
            print('Quota exceeded')
            break
        else:
            request = youtube.commentThreads().list(
                part="snippet",
                maxResults=100,
                pageToken=next_page_token,
                videoId=video
            )
            try:
                response = request.execute()
                quota += 3
                next_page_token, results = get_infos(response)
                video_comments_count+=len(results)
                comments_count+=video_comments_count
                comments.append(results)
            except:
                videos_disabled_comments.append(video)
                next_page_token = None
                print('Video has disabled comments')
    if quota > 9900:
        break
    if next_page_token == None or video_comments_count>=10000:
        if next_page_token != None:
            print('Reached max comment count')
            videos_not_finished.append([video, next_page_token])
        last_video_idx += 1
        next_page_token = ' '
        print('Total comments for {}: {}'.format(video, video_comments_count))

print("Done with {} comments, video to start: {}\n with next_page_token: {}".format(comments_count,last_video_idx, next_page_token))

________________________________________________________
999 video: puxXSTyYh7s
Reached quotas: 0
Reached quotas: 3
Reached quotas: 6
Reached quotas: 9
Reached quotas: 12
Reached quotas: 15
Reached quotas: 18
Reached quotas: 21
Reached quotas: 24
Reached quotas: 27
Reached quotas: 30
Reached quotas: 33
Reached quotas: 36
Reached quotas: 39
Reached quotas: 42
Reached quotas: 45
Reached quotas: 48
Reached quotas: 51
Reached quotas: 54
Reached quotas: 57
Reached quotas: 60
Reached quotas: 63
Reached quotas: 66
Reached quotas: 69
Reached quotas: 72
Reached quotas: 75
Reached quotas: 78
Reached quotas: 81
Reached quotas: 84
Reached quotas: 87
Reached quotas: 90
Reached quotas: 93
Reached quotas: 96
Reached quotas: 99
Reached quotas: 102
Reached quotas: 105
Reached quotas: 108
Reached quotas: 111
Reached quotas: 114
Reached quotas: 117
Reached quotas: 120
Reached quotas: 123
Reached quotas: 126
Reached quotas: 129
Reached quotas: 132
Reached quotas: 135
Reached quotas: 138
Reached quotas: 14

Video has disabled comments
Total comments for AZ0WL4nq9bg: 0
________________________________________________________
1032 video: f9RUFYeUav8
Reached quotas: 477
Video has disabled comments
Total comments for f9RUFYeUav8: 0
________________________________________________________
1033 video: 75AL-XN84cI
Reached quotas: 477
Video has disabled comments
Total comments for 75AL-XN84cI: 0
________________________________________________________
1034 video: q5MTQz1ubGc
Reached quotas: 477
Video has disabled comments
Total comments for q5MTQz1ubGc: 0
________________________________________________________
1035 video: fx_yBYKF3Ck
Reached quotas: 477
Video has disabled comments
Total comments for fx_yBYKF3Ck: 0
________________________________________________________
1036 video: smmeWUo6VAE
Reached quotas: 477
Video has disabled comments
Total comments for smmeWUo6VAE: 0
________________________________________________________
1037 video: 3_gmgYA8Cg0
Reached quotas: 477
Video has disabled com

Reached quotas: 609
Page token non esistente
Total comments for w--jaRdQNx8: 114
________________________________________________________
1082 video: crNaYhfvIzo
Reached quotas: 612
Reached quotas: 615
Reached quotas: 618
Reached quotas: 621
Page token non esistente
Total comments for crNaYhfvIzo: 310
________________________________________________________
1083 video: 9BVPzUwrr80
Reached quotas: 624
Page token non esistente
Total comments for 9BVPzUwrr80: 27
________________________________________________________
1084 video: TH4sMXDa11U
Reached quotas: 627
Page token non esistente
Total comments for TH4sMXDa11U: 41
________________________________________________________
1085 video: N02H82LI7iM
Reached quotas: 630
Reached quotas: 633
Page token non esistente
Total comments for N02H82LI7iM: 178
________________________________________________________
1086 video: weVbIVrqtzs
Reached quotas: 636
Reached quotas: 639
Reached quotas: 642
Page token non esistente
Total comments for weVbIVrq

Reached quotas: 1146
Reached quotas: 1149
Reached quotas: 1152
Reached quotas: 1155
Reached quotas: 1158
Reached quotas: 1161
Reached quotas: 1164
Reached quotas: 1167
Page token non esistente
Total comments for CpqqkZPmd7o: 848
________________________________________________________
1115 video: H2QG8z4rYxw
Reached quotas: 1170
Reached quotas: 1173
Reached quotas: 1176
Reached quotas: 1179
Reached quotas: 1182
Page token non esistente
Total comments for H2QG8z4rYxw: 468
________________________________________________________
1116 video: DvfSJQRvBFs
Reached quotas: 1185
Reached quotas: 1188
Reached quotas: 1191
Reached quotas: 1194
Reached quotas: 1197
Page token non esistente
Total comments for DvfSJQRvBFs: 401
________________________________________________________
1117 video: VnNRQwwuTfM
Reached quotas: 1200
Reached quotas: 1203
Reached quotas: 1206
Reached quotas: 1209
Page token non esistente
Total comments for VnNRQwwuTfM: 393
___________________________________________________

Reached quotas: 1902
Reached quotas: 1905
Reached quotas: 1908
Reached quotas: 1911
Reached quotas: 1914
Reached quotas: 1917
authorChannelId
Problem with commentThread UgzBkyZRLB7ke6WeZZt4AaABAg
Reached quotas: 1920
Reached quotas: 1923
Reached quotas: 1926
Reached quotas: 1929
Page token non esistente
Total comments for jmUmYj4kD88: 1016
________________________________________________________
1136 video: 8yqWjOY4LBY
Reached quotas: 1932
Reached quotas: 1935
Reached quotas: 1938
Reached quotas: 1941
Reached quotas: 1944
Reached quotas: 1947
Reached quotas: 1950
Reached quotas: 1953
Reached quotas: 1956
Reached quotas: 1959
Reached quotas: 1962
Reached quotas: 1965
Reached quotas: 1968
Reached quotas: 1971
Page token non esistente
Total comments for 8yqWjOY4LBY: 1320
________________________________________________________
1137 video: 2xFGitPp0fw
Reached quotas: 1974
Reached quotas: 1977
Reached quotas: 1980
Reached quotas: 1983
Reached quotas: 1986
Reached quotas: 1989
Reached quotas

...

Page token non esistente
Total comments for sRnFaotzFm4: 135
________________________________________________________
1606 video: 1FeCjnF16ko
Reached quotas: 7620
Reached quotas: 7623
Reached quotas: 7626
Reached quotas: 7629
Page token non esistente
Total comments for 1FeCjnF16ko: 399
________________________________________________________
1607 video: 3W8AVe3PmJs
Reached quotas: 7632
Reached quotas: 7635
Reached quotas: 7638
Reached quotas: 7641
Reached quotas: 7644
Reached quotas: 7647
Reached quotas: 7650
Page token non esistente
Total comments for 3W8AVe3PmJs: 640
________________________________________________________
1608 video: iNs-uLfWoKE
Reached quotas: 7653
Reached quotas: 7656
Reached quotas: 7659
Reached quotas: 7662
Reached quotas: 7665
Reached quotas: 7668
Reached quotas: 7671
Reached quotas: 7674
Reached quotas: 7677
Reached quotas: 7680
Reached quotas: 7683
Reached quotas: 7686
Reached quotas: 7689
Reached quotas: 7692
Reached quotas: 7695
Reached quotas: 7698
Reached

Page token non esistente
Total comments for LF4OvXTzHUk: 58
________________________________________________________
1644 video: CpYN3Wde4Po
Reached quotas: 8025
Page token non esistente
Total comments for CpYN3Wde4Po: 57
________________________________________________________
1645 video: UEgVXdDgv4s
Reached quotas: 8028
Page token non esistente
Total comments for UEgVXdDgv4s: 55
________________________________________________________
1646 video: KBtpd-tOdsw
Reached quotas: 8031
Page token non esistente
Total comments for KBtpd-tOdsw: 60
________________________________________________________
1647 video: GEUE3jAlUEs
Reached quotas: 8034
Page token non esistente
Total comments for GEUE3jAlUEs: 31
________________________________________________________
1648 video: paDtBcp85TM
Reached quotas: 8037
Page token non esistente
Total comments for paDtBcp85TM: 68
________________________________________________________
1649 video: 6xG4gX_FgUA
Reached quotas: 8040
Page token non esistente
Tot

Reached quotas: 8736
Reached quotas: 8739
Page token non esistente
Total comments for QRGO7L-jHd8: 275
________________________________________________________
1667 video: SqggoyxVgK0
Reached quotas: 8742
Reached quotas: 8745
Reached quotas: 8748
Reached quotas: 8751
Reached quotas: 8754
Page token non esistente
Total comments for SqggoyxVgK0: 413
________________________________________________________
1668 video: 5Rj6VGzUD_s
Reached quotas: 8757
Reached quotas: 8760
Reached quotas: 8763
Reached quotas: 8766
Reached quotas: 8769
Reached quotas: 8772
Page token non esistente
Total comments for 5Rj6VGzUD_s: 516
________________________________________________________
1669 video: M6YUN5u7Spo
Reached quotas: 8775
Reached quotas: 8778
Reached quotas: 8781
Reached quotas: 8784
Page token non esistente
Total comments for M6YUN5u7Spo: 362
________________________________________________________
1670 video: h8tZP8qtArE
Reached quotas: 8787
Reached quotas: 8790
Reached quotas: 8793
Page token n

Reached quotas: 9501
Reached quotas: 9504
Page token non esistente
Total comments for SQiA8aT7sEE: 1481
________________________________________________________
1687 video: _uI3uqBmq0o
Reached quotas: 9507
Reached quotas: 9510
Reached quotas: 9513
Reached quotas: 9516
Page token non esistente
Total comments for _uI3uqBmq0o: 355
________________________________________________________
1688 video: EYPy7DQais4
Reached quotas: 9519
Reached quotas: 9522
Reached quotas: 9525
Reached quotas: 9528
Reached quotas: 9531
Reached quotas: 9534
Reached quotas: 9537
Reached quotas: 9540
Reached quotas: 9543
Page token non esistente
Total comments for EYPy7DQais4: 814
________________________________________________________
1689 video: xW2tcNqBe-0
Reached quotas: 9546
Reached quotas: 9549
Reached quotas: 9552
Reached quotas: 9555
Reached quotas: 9558
Reached quotas: 9561
Page token non esistente
Total comments for xW2tcNqBe-0: 530
________________________________________________________
1690 video: --

Reached quotas: 9861
Reached quotas: 9864
Video has disabled comments
Total comments for MNhWouEXCi8: 200
________________________________________________________
1726 video: d4_YGZcmF1I
Reached quotas: 9864
Reached quotas: 9867
Reached quotas: 9870
Page token non esistente
Total comments for d4_YGZcmF1I: 246
________________________________________________________
1727 video: eEiPlSAopDM
Reached quotas: 9873
Video has disabled comments
Total comments for eEiPlSAopDM: 0
________________________________________________________
1728 video: DdMhWjKtEoo
Reached quotas: 9873
Reached quotas: 9876
Reached quotas: 9879
Page token non esistente
Total comments for DdMhWjKtEoo: 214
________________________________________________________
1729 video: ZA_jAJ9brzU
Reached quotas: 9882
Reached quotas: 9885
Video has disabled comments
Total comments for ZA_jAJ9brzU: 100
________________________________________________________
1730 video: r6JIY-5Jnxs
Reached quotas: 9885
Video has disabled comments
Tot

In [14]:
np.save('data/get_comment_threads_output/next_page_token.npy',next_page_token)
np.save('data/get_comment_threads_output/video_to_start.npy',last_video_idx)
np.save('data/get_comment_threads_output/videos_not_finished.npy',videos_not_finished)
np.save('data/get_comment_threads_output/videos_disabled_comments.npy',videos_disabled_comments)

In [15]:
print(next_page_token, last_video_idx)

QURTSl9pMGVXd2RTZVc4TTFHRUZaYWowYUkxc21qYTRHdGpkWUd3dlFpazFnaTl3NTc1RVE5dGJ6RVNQTkJEaVdncXExc2w2U1VUbUJvOUo3enJuR25rY0lSOUVJWmNmY0I0QjZfaXMwN0NLMjYtdDd6RzRHMWlpZjVrNDFEU1lyalU= 1733


In [16]:
format_ok_comments = []
for outer_comment in comments:
    for comment in outer_comment:
        format_ok_comments.append(comment)

In [17]:
format_ok_comments

[['puxXSTyYh7s',
  'Ugzc6WLVHCReyTS7SRx4AaABAg',
  'Ugzc6WLVHCReyTS7SRx4AaABAg',
  'S M',
  'UCRTTNWxFOCo8DPDR26rCp_Q',
  'Row, row, row your boat gently down the stream. Merrily, merrily, merrily, merrily life is but a dreaaaaaaaaaaaam.'],
 ['puxXSTyYh7s',
  'UgxCT41BWJIlfjqsMTh4AaABAg',
  'UgxCT41BWJIlfjqsMTh4AaABAg',
  'God bless',
  'UCNmQGq-g_hnZCONMYv9K_pA',
  'Repent and trust in Christ before it’s too late! God bless you'],
 ['puxXSTyYh7s',
  'UgzNreVOmRwN8ZLMS0F4AaABAg',
  'UgzNreVOmRwN8ZLMS0F4AaABAg',
  'G-ful Nar',
  'UC9UnxL-uY5Ly5y01CaVCiUQ',
  "Man her Youtube career really is ruined! Guess she's gonna have to go to college to find a new career path...oh wait!"],
 ['puxXSTyYh7s',
  'UgyejVcOPz8c39wqT7F4AaABAg',
  'UgyejVcOPz8c39wqT7F4AaABAg',
  'Vicky Iglesias',
  'UCk_sMM2FfOYu_2oylWjuA1A',
  'I still love u idc <3'],
 ['puxXSTyYh7s',
  'UgxIbTZTUFhMY2ujJJZ4AaABAg',
  'UgxIbTZTUFhMY2ujJJZ4AaABAg',
  'Melanie',
  'UChFinlCMlb6noS-1PWlJRHQ',
  'You missed the part of the c

In [18]:
df_comment_thread = pd.DataFrame(format_ok_comments,columns=['video_id','comment_thread_id','top_level_comment_id','author_display_name','author_channel_id','text_original'])

In [19]:
df_comment_thread

,video_id,comment_thread_id,top_level_comment_id,author_display_name,author_channel_id,text_original
0,puxXSTyYh7s,Ugzc6WLVHCReyTS7SRx4AaABAg,Ugzc6WLVHCReyTS7SRx4AaABAg,S M,UCRTTNWxFOCo8DPDR26rCp_Q,"Row, row, row your boat gently down the stream..."
1,puxXSTyYh7s,UgxCT41BWJIlfjqsMTh4AaABAg,UgxCT41BWJIlfjqsMTh4AaABAg,God bless,UCNmQGq-g_hnZCONMYv9K_pA,Repent and trust in Christ before it’s too lat...
2,puxXSTyYh7s,UgzNreVOmRwN8ZLMS0F4AaABAg,UgzNreVOmRwN8ZLMS0F4AaABAg,G-ful Nar,UC9UnxL-uY5Ly5y01CaVCiUQ,Man her Youtube career really is ruined! Guess...
3,puxXSTyYh7s,UgyejVcOPz8c39wqT7F4AaABAg,UgyejVcOPz8c39wqT7F4AaABAg,Vicky Iglesias,UCk_sMM2FfOYu_2oylWjuA1A,I still love u idc <3
4,puxXSTyYh7s,UgxIbTZTUFhMY2ujJJZ4AaABAg,UgxIbTZTUFhMY2ujJJZ4AaABAg,Melanie,UChFinlCMlb6noS-1PWlJRHQ,You missed the part of the cheek filler? How d...
...,...,...,...,...,...,...
298740,96Hl-mEcnWQ,UgyC0awvM9I95s4Xtj14AaABAg,UgyC0awvM9I95s4Xtj14AaABAg,Curlytashh,UC2Gsofe74b_RSrju-gKI4BA,Just so beautiful 😭❤️
298741,96Hl-mEcnWQ,UgzEU6SdYBes45PWEal4AaABAg,UgzEU6SdYBes45PWEal4AaABAg,Kim van Veen,UCWp9lxX2jTIbY1E8WwgVVDw,Youre Such a beauty!
298742,96Hl-mEcnWQ,UgwhY3cxDZnY9yNSgTd4AaABAg,UgwhY3cxDZnY9yNSgTd4AaABAg,NY’FIAH,UCrh-rvId1Y_rrvheEnna3tA,Yesss I loveeee it 😍😍😍 super perfect for vacat...
298743,96Hl-mEcnWQ,UgzE1nFGYtukXKoFUqV4AaABAg,UgzE1nFGYtukXKoFUqV4AaABAg,Andie Xulu,UCHHVM-Wi_0gohZS30y0yJxg,Those ardell lashes though😍 such a bomb video


In [20]:
last_csv

2

In [21]:
last_csv+=1
last_csv

3

In [22]:
df_comment_thread.to_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(last_csv), index = False)

In [23]:
np.save('data/get_comment_threads_output/last_csv.npy',last_csv)

In [24]:
df = pd.read_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(last_csv))
df

,video_id,comment_thread_id,top_level_comment_id,author_display_name,author_channel_id,text_original
0,puxXSTyYh7s,Ugzc6WLVHCReyTS7SRx4AaABAg,Ugzc6WLVHCReyTS7SRx4AaABAg,S M,UCRTTNWxFOCo8DPDR26rCp_Q,"Row, row, row your boat gently down the stream..."
1,puxXSTyYh7s,UgxCT41BWJIlfjqsMTh4AaABAg,UgxCT41BWJIlfjqsMTh4AaABAg,God bless,UCNmQGq-g_hnZCONMYv9K_pA,Repent and trust in Christ before it’s too lat...
2,puxXSTyYh7s,UgzNreVOmRwN8ZLMS0F4AaABAg,UgzNreVOmRwN8ZLMS0F4AaABAg,G-ful Nar,UC9UnxL-uY5Ly5y01CaVCiUQ,Man her Youtube career really is ruined! Guess...
3,puxXSTyYh7s,UgyejVcOPz8c39wqT7F4AaABAg,UgyejVcOPz8c39wqT7F4AaABAg,Vicky Iglesias,UCk_sMM2FfOYu_2oylWjuA1A,I still love u idc <3
4,puxXSTyYh7s,UgxIbTZTUFhMY2ujJJZ4AaABAg,UgxIbTZTUFhMY2ujJJZ4AaABAg,Melanie,UChFinlCMlb6noS-1PWlJRHQ,You missed the part of the cheek filler? How d...
...,...,...,...,...,...,...
298740,96Hl-mEcnWQ,UgyC0awvM9I95s4Xtj14AaABAg,UgyC0awvM9I95s4Xtj14AaABAg,Curlytashh,UC2Gsofe74b_RSrju-gKI4BA,Just so beautiful 😭❤️
298741,96Hl-mEcnWQ,UgzEU6SdYBes45PWEal4AaABAg,UgzEU6SdYBes45PWEal4AaABAg,Kim van Veen,UCWp9lxX2jTIbY1E8WwgVVDw,Youre Such a beauty!
298742,96Hl-mEcnWQ,UgwhY3cxDZnY9yNSgTd4AaABAg,UgwhY3cxDZnY9yNSgTd4AaABAg,NY’FIAH,UCrh-rvId1Y_rrvheEnna3tA,Yesss I loveeee it 😍😍😍 super perfect for vacat...
298743,96Hl-mEcnWQ,UgzE1nFGYtukXKoFUqV4AaABAg,UgzE1nFGYtukXKoFUqV4AaABAg,Andie Xulu,UCHHVM-Wi_0gohZS30y0yJxg,Those ardell lashes though😍 such a bomb video
